In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [3]:
np.random.seed(2018)

trn = pd.read_csv('/home/jeongchanwoo/workspace/git/study/Kaggle_data/santander-product-recommendation/input/train_ver2.csv')
tst = pd.read_csv('/home/jeongchanwoo/workspace/git/study/Kaggle_data/santander-product-recommendation/input/test_ver2.csv')

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
## 데이터 전처리 ##

# 제품 변수
prods = trn.columns[24:].tolist()

# 제품 변수 결측값 0으로 대체
trn[prods] = trn[prods].fillna(0).astype(np.int8)

In [5]:
#24개 제품 중 하나도 보유하지 않은 고객 제거
no_product = trn[prods].sum(axis=1) ==0
trn = trn[~no_product]

In [6]:
# 훈련 데이터 테스트 데이터 통합 테스트 데이터에 없는 제품 변수 0으로 채움
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis = 0)

In [7]:
# 학습 변수 list
features = []


In [8]:
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp','canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']

In [9]:
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel = -99)
features += categorical_cols

In [10]:
df['renta'][929602].values

array([137940.75, '         NA'], dtype=object)

In [11]:
df.renta.unique()

array([87218.1, 35548.74, 122179.11000000002, ..., '  139164.12',
       '  100647.45', '   72765.27'], dtype=object)

In [12]:
# 수치형 변수의 특이값, 결측값 -99로 대체하고 정수형 변환
df['age'].replace(' NA', -99, inplace = True)
df['age']  = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace = True)
df['antiguedad']  = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace = True)
df['renta'].fillna(-99, inplace=True)
df['renta']  = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace = True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes']  = df['indrel_1mes'].astype(float).astype(np.int8)

In [13]:
# 학습에 사용되는 수치형 변수 features에 추가
features += ['age' ,'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']

## Feature Engineering

In [14]:
# 날짜 변수에서 연도와 월 정보 추출
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x : 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [15]:
# 그외 변수의 결측값은 -99로 대체
df.fillna(-99, inplace=True)

In [16]:
# lag-1 데이터 생성

# 날짜를 숫자로 변환
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date


# 날짜를 숫자로 변환해 int_date에 저장
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

#데이터 카피하고 , int_date 날짜에 1을 더해 lag를 생성, 변수명에 _prev 추가
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

In [17]:
#원본과 lag데이터를 ncodper와 int_date기준으로 합침
df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how = 'left')

In [18]:
del df, df_lag

In [19]:
# 전 달의 제품 정보가 존재 하지 않을시 0으로 대체
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace = True)

df_trn.fillna(-99, inplace=True)

In [20]:
#lag - 1 변수 추가
features +=  [feature + '_prev' for feature in features]
features +=  [prod + '_prev' for prod in prods]


###
### 추가 피쳐 엔지니어링
###

In [21]:
features

['ind_empleado',
 'pais_residencia',
 'sexo',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'tipodom',
 'nomprov',
 'segmento',
 'age',
 'antiguedad',
 'renta',
 'ind_nuevo',
 'indrel',
 'indrel_1mes',
 'ind_actividad_cliente',
 'fecha_alta_month',
 'fecha_alta_year',
 'ult_fec_cli_1t_month',
 'ult_fec_cli_1t_year',
 'ind_empleado_prev',
 'pais_residencia_prev',
 'sexo_prev',
 'tiprel_1mes_prev',
 'indresi_prev',
 'indext_prev',
 'conyuemp_prev',
 'canal_entrada_prev',
 'indfall_prev',
 'tipodom_prev',
 'nomprov_prev',
 'segmento_prev',
 'age_prev',
 'antiguedad_prev',
 'renta_prev',
 'ind_nuevo_prev',
 'indrel_prev',
 'indrel_1mes_prev',
 'ind_actividad_cliente_prev',
 'fecha_alta_month_prev',
 'fecha_alta_year_prev',
 'ult_fec_cli_1t_month_prev',
 'ult_fec_cli_1t_year_prev',
 'ind_ahor_fin_ult1_prev',
 'ind_aval_fin_ult1_prev',
 'ind_cco_fin_ult1_prev',
 'ind_cder_fin_ult1_prev',
 'ind_cno_fin_ult1_prev',
 'ind_ctju_fin_ult1_prev',
 'ind_ctma_fin_

## 교차검증 데이터 분리

In [22]:
# 2016-01-28 ~ 2016-04-28데이터를 학습에 사용하고
# 검증에 2016-05-28 데이터를 사용한다.

use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn




In [23]:
# 훈련 데이터에서 신규 구매 건수만 추출
X = []
Y = []

for i,prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0 )] # 상품 구매자
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i #상품 숫자(LE)
    X.append(prX)
    Y.append(prY)

In [24]:
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [25]:
# 훈련 검증 데이터 분리
vld_date = '2016-05-28'
xy_trn = XY[XY['fecha_dato'] != vld_date]
xy_vld = XY[XY['fecha_dato'] == vld_date]

## XGBoost 학습

In [26]:
param = {
    'booster' : 'gbtree',
    'max_depth' : 8,
    'nthread' : -1,
    'num_class' : len(prods),
    'objective' : 'multi:softprob',
    'silent' : 1,
    'eval_metric' : 'mlogloss',
    'eta' : 0.1,
    'min_child_weight' : 10,
    'colsample_bytree' : 0.8,
    'colsample_bylevel' : 0.9,
    'seed' : 2018,
    'max_bin': 16, # GPU 
    'tree_method': 'gpu_hist', # GPU method (자세한 설명은 문서로!)
    'predictor':'gpu_predictor' # train뿐만아니라 predict할때도 gpu쓸건지 결정
    
}
# XGBoost 형태로 훈련, 검증 데이터 변환
X_trn = xy_trn.as_matrix(columns=features)
Y_trn = xy_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = xy_vld.as_matrix(columns=features)
Y_vld = xy_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)




/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [27]:
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

[0]	train-mlogloss:2.75913	eval-mlogloss:2.784
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.53812	eval-mlogloss:2.5674
[2]	train-mlogloss:2.37132	eval-mlogloss:2.40377
[3]	train-mlogloss:2.21234	eval-mlogloss:2.24439
[4]	train-mlogloss:2.08782	eval-mlogloss:2.11911
[5]	train-mlogloss:1.98199	eval-mlogloss:2.01291
[6]	train-mlogloss:1.89548	eval-mlogloss:1.92643
[7]	train-mlogloss:1.82052	eval-mlogloss:1.85058
[8]	train-mlogloss:1.75791	eval-mlogloss:1.78776
[9]	train-mlogloss:1.70137	eval-mlogloss:1.73079
[10]	train-mlogloss:1.65118	eval-mlogloss:1.68072
[11]	train-mlogloss:1.60467	eval-mlogloss:1.63358
[12]	train-mlogloss:1.56373	eval-mlogloss:1.59332
[13]	train-mlogloss:1.52618	eval-mlogloss:1.55517
[14]	train-mlogloss:1.49251	eval-mlogloss:1.52156
[15]	train-mlogloss:1.46185	eval-mlogloss:1.49053
[16]	train-mlogloss:1.43552	eval-mlogloss:1.46422
[17]	train-m

[161]	train-mlogloss:1.01798	eval-mlogloss:1.10385
[162]	train-mlogloss:1.01726	eval-mlogloss:1.10384
[163]	train-mlogloss:1.01673	eval-mlogloss:1.10384
[164]	train-mlogloss:1.01627	eval-mlogloss:1.10381
[165]	train-mlogloss:1.01567	eval-mlogloss:1.10375
[166]	train-mlogloss:1.01508	eval-mlogloss:1.10381
[167]	train-mlogloss:1.01456	eval-mlogloss:1.10378
[168]	train-mlogloss:1.01397	eval-mlogloss:1.10375
[169]	train-mlogloss:1.01342	eval-mlogloss:1.10371
[170]	train-mlogloss:1.01292	eval-mlogloss:1.10367
[171]	train-mlogloss:1.0125	eval-mlogloss:1.10367
[172]	train-mlogloss:1.01197	eval-mlogloss:1.10369
[173]	train-mlogloss:1.01141	eval-mlogloss:1.1037
[174]	train-mlogloss:1.01084	eval-mlogloss:1.10374
[175]	train-mlogloss:1.01021	eval-mlogloss:1.10372
[176]	train-mlogloss:1.00946	eval-mlogloss:1.1037
[177]	train-mlogloss:1.009	eval-mlogloss:1.10367
[178]	train-mlogloss:1.00847	eval-mlogloss:1.10361
[179]	train-mlogloss:1.00809	eval-mlogloss:1.1036
[180]	train-mlogloss:1.00746	eval-mlo

In [28]:
import pickle

In [29]:
pickle.dump(model, open("./model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit

## 교차검증

In [30]:
#고객 식별 번호 추출
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns = ['ncodpers'])

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
# 검증 데이터에서 신규 구매를 구함
for prod in prods:
    prev = prod +'_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]
    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [32]:
# 고객별 신규 구매 정답값을 add_vld_list에 저장, 총 count를 count_vld에 저장
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

In [33]:
from mapk import mapk

In [34]:
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

0.04266379915553903


In [35]:
# 검증 데이터에 대한 예측 값
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names= features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)


/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
# 저번 달 보유한 제품은 신규구매 불가이기에 확률값에서 미리 1을 빼줌
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [37]:
#검증 데이터 예측 상위 7개
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p, ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a : a[0], reverse=True)[:7]
    result_vld.append([ip for y,p, ip in y_prods])

In [38]:
# 검증 데이터에서 MAP@7 점수 구함
print(mapk(add_vld_list,result_vld, 7, 0.0))

0.036266311174317035


# 캐글 제출

In [39]:
# XGBoost 모델을 전체 훈련 데이터로 재학습
X_all  = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall  = xgb.DMatrix(X_all, label=Y_all, feature_names=features)

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
watch_list = [(dall, 'train')]

In [41]:
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가
best_ntree_limit = int(best_ntree_limit * (len(xy_trn) + len(xy_vld))/ len(xy_trn) )


In [42]:
best_ntree_limit

251

In [43]:
#XGBoost 모델 재학습
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

[0]	train-mlogloss:2.76102
[1]	train-mlogloss:2.53979
[2]	train-mlogloss:2.34682
[3]	train-mlogloss:2.19149
[4]	train-mlogloss:2.07457
[5]	train-mlogloss:1.98141
[6]	train-mlogloss:1.89429
[7]	train-mlogloss:1.81817
[8]	train-mlogloss:1.75744
[9]	train-mlogloss:1.69884
[10]	train-mlogloss:1.64685
[11]	train-mlogloss:1.60126
[12]	train-mlogloss:1.56056
[13]	train-mlogloss:1.52531
[14]	train-mlogloss:1.49293
[15]	train-mlogloss:1.46454
[16]	train-mlogloss:1.43677
[17]	train-mlogloss:1.41226
[18]	train-mlogloss:1.39071
[19]	train-mlogloss:1.36975
[20]	train-mlogloss:1.35069
[21]	train-mlogloss:1.33344
[22]	train-mlogloss:1.31765
[23]	train-mlogloss:1.30279
[24]	train-mlogloss:1.28822
[25]	train-mlogloss:1.27598
[26]	train-mlogloss:1.26338
[27]	train-mlogloss:1.25253
[28]	train-mlogloss:1.24163
[29]	train-mlogloss:1.23178
[30]	train-mlogloss:1.22251
[31]	train-mlogloss:1.21429
[32]	train-mlogloss:1.20645
[33]	train-mlogloss:1.19877
[34]	train-mlogloss:1.19138
[35]	train-mlogloss:1.18447
[3

In [44]:
# 변수 중요도 출력
print("Featur importance :")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key = lambda kv : kv[1], reverse=True):
    print(kv)

Featur importance :
('fecha_alta_month', 17543)
('age', 16408)
('renta', 14951)
('fecha_alta_year', 12555)
('age_prev', 12501)
('nomprov', 11892)
('antiguedad', 10775)
('antiguedad_prev', 9556)
('canal_entrada', 8979)
('renta_prev', 8076)
('fecha_alta_month_prev', 7846)
('nomprov_prev', 7609)
('canal_entrada_prev', 6222)
('fecha_alta_year_prev', 6046)
('sexo', 5000)
('ind_recibo_ult1_prev', 4599)
('ind_ecue_fin_ult1_prev', 4577)
('ind_cco_fin_ult1_prev', 3974)
('ind_cno_fin_ult1_prev', 3686)
('segmento', 3548)
('ind_reca_fin_ult1_prev', 3244)
('segmento_prev', 3238)
('ind_tjcr_fin_ult1_prev', 3150)
('tiprel_1mes', 2301)
('ind_valo_fin_ult1_prev', 2271)
('sexo_prev', 2264)
('ind_ctop_fin_ult1_prev', 2261)
('ind_nom_pens_ult1_prev', 2259)
('ind_dela_fin_ult1_prev', 2197)
('ind_nomina_ult1_prev', 1909)
('ind_ctpp_fin_ult1_prev', 1893)
('tiprel_1mes_prev', 1696)
('ind_actividad_cliente', 1690)
('ind_fond_fin_ult1_prev', 1490)
('ind_nuevo', 1324)
('ind_actividad_cliente_prev', 1324)
('ind_c

In [45]:
# 테스트 데이터 예측값
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns = [prod + '_prev' for prod in prods])

/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
/home/jeongchanwoo/miniconda3/envs/ml_python_linux/lib/python3.5/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [47]:
preds_tst

(929615, 24)

In [67]:
ncodpers_tst[0].item()

15889

In [96]:
# submit_file = open('./model/xgb.baseline.2015-06-28', 'w')
# submit_file.write('ncodpers, added_products\n')
blank_dict = {}
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key = lambda a : a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
#     submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
    y_prods = ' '.join(y_prods)
    blank_dict[ncodper.item()] = [y_prods]

In [97]:
blank_dict

{15889: ['ind_recibo_ult1 ind_ecue_fin_ult1 ind_nomina_ult1 ind_nom_pens_ult1 ind_ctop_fin_ult1 ind_cno_fin_ult1 ind_reca_fin_ult1'],
 15890: ['ind_valo_fin_ult1 ind_fond_fin_ult1 ind_cco_fin_ult1 ind_reca_fin_ult1 ind_dela_fin_ult1 ind_ctop_fin_ult1 ind_hip_fin_ult1'],
 15892: ['ind_nom_pens_ult1 ind_cno_fin_ult1 ind_nomina_ult1 ind_ctpp_fin_ult1 ind_fond_fin_ult1 ind_plan_fin_ult1 ind_ctop_fin_ult1'],
 15893: ['ind_cco_fin_ult1 ind_tjcr_fin_ult1 ind_ecue_fin_ult1 ind_recibo_ult1 ind_nom_pens_ult1 ind_cno_fin_ult1 ind_dela_fin_ult1'],
 15894: ['ind_cno_fin_ult1 ind_plan_fin_ult1 ind_ctpp_fin_ult1 ind_hip_fin_ult1 ind_fond_fin_ult1 ind_dela_fin_ult1 ind_ctop_fin_ult1'],
 15895: ['ind_nom_pens_ult1 ind_cno_fin_ult1 ind_nomina_ult1 ind_ctpp_fin_ult1 ind_ctop_fin_ult1 ind_dela_fin_ult1 ind_fond_fin_ult1'],
 15896: ['ind_recibo_ult1 ind_tjcr_fin_ult1 ind_reca_fin_ult1 ind_nom_pens_ult1 ind_cno_fin_ult1 ind_dela_fin_ult1 ind_nomina_ult1'],
 15897: ['ind_nomina_ult1 ind_cco_fin_ult1 ind_dela

In [104]:
submit = pd.DataFrame.from_dict(blank_dict, columns= ['added_products'], orient='index')

In [107]:
submit.rename_axis("ncodpers",inplace=True)
submit.head()

,added_products
ncodpers,
15889,ind_recibo_ult1 ind_ecue_fin_ult1 ind_nomina_u...
15890,ind_valo_fin_ult1 ind_fond_fin_ult1 ind_cco_fi...
15892,ind_nom_pens_ult1 ind_cno_fin_ult1 ind_nomina_...
15893,ind_cco_fin_ult1 ind_tjcr_fin_ult1 ind_ecue_fi...
15894,ind_cno_fin_ult1 ind_plan_fin_ult1 ind_ctpp_fi...


In [108]:
submit.to_csv('./model/xgb.baseline.2015-06-28')